In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, activations, initializers, losses, optimizers, metrics

import os
tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # https://stackoverflow.com/questions/35911252/disable-tensorflow-debugging-information

/Users/chaeyoon/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/chaeyoon/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/chaeyoon/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/chaeyoon/opt/anaconda3/lib/python3.7/site-packages/ten

<br> 

#### 1. Prepare train & test data (MNIST)

#### Train & Test split

In [3]:
(train_data, train_label), (test_data, test_label) = datasets.mnist.load_data()

print(train_data.shape) # # of training data == 60000, each data = 28px * 28px
print(test_data.shape) # # of test data == 10000
print(train_data.dtype) # int
print(train_data.max()) # 0~255

(60000, 28, 28)
(10000, 28, 28)
uint8
255


#### Normalization

In [4]:
train_data = train_data.reshape(60000, 784).astype('float32') / 255.0 # 0~255 -> 0~1
test_data = test_data.reshape(10000, 784).astype('float32') / 255.0 # 0~255 -> 0~1

#### One-hot encoding

In [5]:
train_label = utils.to_categorical(train_label) # 0~9 -> one-hot vector
test_label = utils.to_categorical(test_label) # 0~9 -> one-hot vector

<br> 

#### 2. Build the model & Set the criterion

In [6]:
model = models.Sequential() 

model.add(layers.Dense(input_dim=28*28, units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu')) # layers.ELU or layers.LeakyReLU
model.add(layers.Dropout(rate=0.2))

model.add(layers.Dense(units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu')) # layers.ELU or layers.LeakyReLU
model.add(layers.Dropout(rate=0.2))

model.add(layers.Dense(units=10, activation='softmax')) # 0~9 

In [7]:
model.compile(optimizer=optimizers.Adam(), 
              loss=losses.categorical_crossentropy, 
              metrics=[metrics.categorical_accuracy]) 

<br> 

#### 3. Train the model

In [8]:
history = model.fit(train_data, train_label, batch_size=100, epochs=15, validation_split=0.2) 

Train on 48000 samples, validate on 12000 samples
Epoch 1/15
48000/48000 [==============================] - 17s 347us/sample - loss: 0.3043 - categorical_accuracy: 0.9083 - val_loss: 0.1233 - val_categorical_accuracy: 0.9639
Epoch 2/15
48000/48000 [==============================] - 14s 298us/sample - loss: 0.1416 - categorical_accuracy: 0.9569 - val_loss: 0.1020 - val_categorical_accuracy: 0.9693
Epoch 3/15
48000/48000 [==============================] - 12s 251us/sample - loss: 0.1028 - categorical_accuracy: 0.9678 - val_loss: 0.0839 - val_categorical_accuracy: 0.9746
Epoch 4/15
48000/48000 [==============================] - 8s 170us/sample - loss: 0.0828 - categorical_accuracy: 0.9740 - val_loss: 0.0870 - val_categorical_accuracy: 0.9743
Epoch 5/15
48000/48000 [==============================] - 8s 170us/sample - loss: 0.0697 - categorical_accuracy: 0.9778 - val_loss: 0.0770 - val_categorical_accuracy: 0.9781
Epoch 6/15
48000/48000 [==============================] - 8s 173us/sample - l

<br> 

#### 4. Test the model

In [9]:
result = model.evaluate(test_data, test_label, batch_size=100)

print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

10000/10000 [==============================] - 0s 34us/sample - loss: 0.0712 - categorical_accuracy: 0.9809
loss (cross-entropy) : 0.0711962073620998
test accuracy : 0.9809


<br> 

#### 5. Visualize the result

In [10]:
history.history.keys()

dict_keys(['loss', 'categorical_accuracy', 'val_loss', 'val_categorical_accuracy'])

In [ ]:
val_acc = history.history['val_categorical_accuracy']
acc = history.history['categorical_accuracy']

import numpy as np
import matplotlib.pyplot as plt

x_len = np.arange(len(acc))
plt.plot(x_len, acc, marker='.', c='blue', label="Train-set Acc.")
plt.plot(x_len, val_acc, marker='.', c='red', label="Validation-set Acc.")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.show()